In [39]:
from dotenv import load_dotenv
from anthropic import Anthropic

import os

load_dotenv(os.path.abspath(os.path.join(os.getcwd(), "../.env")))

client = Anthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))

def get_messages(prompt: str, tools = []) -> str:
    response = client.messages.create(
        model="claude-3-5-sonnet-20241022",
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=1024 * 1,
        temperature=0,
        tools=tools,
    )

    return response

In [ ]:
def least_to_most_prompts(question: str) -> str:
    decomposed_questions = decompose_question(question)

    result = []
    for i, q in enumerate(decomposed_questions):
        print(f"decompose_question-{i+1}: {q}\n")
        answer = solve(q, result, question)
        print(f"decompose_answer-{i+1}: {answer}\n")
        result.append(answer)

    return result

def decompose_question(question: str) -> list[str]:
    prompt = f"""
    다음 <question> 태그 안의 질문을 순차적으로 해결하기위해 필요한 하위 질문으로 분해하세요.

    <question>
    {question}
    </question>
    """

    tools = [{
        "name": "decomposition",
        "description": "decomposition for questions",
        "input_schema": {
            "type": "object",
            "properties": {
                "sub_questions": {
                    "type": "array",
                    "items": {
                        "type": "string",
                        "description": "decomposition for questions"
                    }
                }
            },
            "required": ["sub_questions"]
        }
    }]
    response = get_messages(prompt, tools)

    for c in response.content:
        if c.type == "tool_use" and c.name == "decomposition":
            return c.input["sub_questions"]

    return []

def solve(question: str, answers: list[str], full_question: str) -> str:

    joined_answers = "\n".join(answers)  # 개행 문자로 구분

    prompt = f"""
    <full_question> 태그 안에 있는 내용은 원래 해결하고자 했던 원본 질문이고
    <answers> 태그 안에 있는 내용은 원본 질문을 해결하기 위해 분해했던, 하위 질문들 중 이미 해결한 답변입니다.

    <full_question>
    {full_question}
    </full_question>

    <answers>
    {joined_answers}
    <answers>

    이를 참고하여 이제 <question> 태그 안의 하위 질문에 대해 답변해주세요.

    <question>
    {question}
    </question>
    """

    return get_messages(prompt).content[0].text

In [ ]:
print(least_to_most_prompts("""
사용자가 쇼핑몰에서 특정 브랜드의 신발을 검색할 때, 가격, 재고 여부 등을 고려하여 가장 적합한 추천 상품을 제공하는 방법을 설계하시오.
"""))

In [40]:
import asyncio
import nest_asyncio
from anthropic import AsyncAnthropic

async_client = AsyncAnthropic(api_key=os.getenv("ANTHROPIC_API_KEY"))
nest_asyncio.apply()

async def async_get_message(prompt, tools):
    return await async_client.messages.create(
        model="claude-3-5-sonnet-20241022",
        messages=[
            {"role": "user", "content": prompt}
        ],
        max_tokens=1024 * 1,
        temperature=0,
        tools=tools,
    )

async def get_skeletons(question: str) -> list[str]:
    prompt = f"""
    당신은 질문에 대한 답변이 아닌, 구체적인 뼈대(skeletons)만을 제공하는 역할입니다.
    다음은 뼈대를 작성하는 방식에 대한 지침입니다.
    1. 질문에 답하기 위해 숫자형 리스트(1,2,3,4)로 구성하세요.
    2. 각 항목은 간결하게 작성해야 합니다.
    3. 일반적으로 3~10개의 항목으로 구성합니다.
    4. 완전한 문장보다는 간결하고 핵심적인 짧은 요약 표현을 사용합니다.

    아래 <question> 태그 안의 질문에 대한 뼈대를 작성해주세요.

    <question>
    {question}
    </question>
    """

    tools = [{
        "name": "SoT",
        "description": "skeletons for questions",
        "input_schema": {
            "type": "object",
            "properties": {
                "skeletons": {
                    "type": "array",
                    "items": {
                        "type": "string",
                        "description": "skeletons for questions"
                    }
                }
            },
            "required": ["skeletons"]
        }
    }]
    response = await async_get_message(prompt, tools)
    for c in response.content:
        if c.type == "tool_use" and c.name == "SoT":
            return c.input["skeletons"]

    return []

async def expand_point(question, skeletons, point_index):
    prompt = f"""
    당신은 <question> 태그 안의 질문에 대한 전체 답변 중 하나의 항목만 이어서 작성하는 역할을 맡고 있습니다.

    <question>
    {question}
    </question>

    이제 다음 <skeleton> 태그 안의 뼈대를 참고해서, 해당 항목만 이어서 작성해주세요.
    작성해야할 항목은  숫자형 리스트 중 {point_index}번 항목입니다.
    답변은 핵심만 간결하고 짧게 작성해주세요.
    절대 다른 항목에 대한 답변을 작성하면 안됩니다.

    <skeleton>
    {skeletons}
    </skeleton>
    """

    return (await async_get_message(prompt, [])).content[0].text.replace("\n", " ").strip()

async def skeleton_of_thought(question: str) -> str:
    skeletons = await get_skeletons(question)
    print(f"skeletons: {skeletons}\n")

    expanded_points = [expand_point(question, skeletons, point_index + 1) for point_index in range(len(skeletons))]

    response = await asyncio.gather(*expanded_points)
    return "\n\n".join(response)


In [41]:
print(asyncio.run(skeleton_of_thought("오믈렛의 종류와 만드는 방법에 대해서 알려줘.")))

Message(id='msg_01GAsV7vb679htU4jszq5mCD', content=[TextBlock(text='오믈렛의 종류와 만드는 방법에 대한 뼈대를 구성하겠습니다.', type='text'), ToolUseBlock(id='toolu_01FVNMQ1Abf9T1RUbGiVbn9H', input={'skeletons': ['1. 기본 재료: 달걀, 소금, 버터, 우유', '2. 오믈렛 종류: 플레인, 치즈, 버섯, 햄, 스페인식', '3. 달걀 풀기: 거품 정도와 우유 비율', '4. 팬 예열: 적정 온도와 버터 사용법', '5. 조리 과정: 달걀물 붓기, 익히기, 접기', '6. 속재료 추가: 타이밍과 배치 방법', '7. 완성 및 플레이팅: 접기 방법, 가니쉬']}, name='SoT', type='tool_use')], model='claude-3-5-sonnet-20241022', role='assistant', stop_reason='tool_use', stop_sequence=None, type='message', usage=Usage(cache_creation_input_tokens=0, cache_read_input_tokens=0, input_tokens=692, output_tokens=297))
skeletons: ['1. 기본 재료: 달걀, 소금, 버터, 우유', '2. 오믈렛 종류: 플레인, 치즈, 버섯, 햄, 스페인식', '3. 달걀 풀기: 거품 정도와 우유 비율', '4. 팬 예열: 적정 온도와 버터 사용법', '5. 조리 과정: 달걀물 붓기, 익히기, 접기', '6. 속재료 추가: 타이밍과 배치 방법', '7. 완성 및 플레이팅: 접기 방법, 가니쉬']

기본 재료: 달걀 2-3개, 소금 약간, 버터 1큰술, 우유 2큰술이 필요합니다.

2. 오믈렛 종류: 플레인(기본형), 치즈(모짜렐라/체다), 버섯(양송이/표고), 햄&치즈, 스페인식(감자/양파)

달걀을 볼에 깨서 넣고 소금을 약간 넣은 후, 거품기로 달걀이 균일하